## Gradient Boosting Classifier
This notebook defines the python script corresponding to the non-DL scenario. The multinomial classifier I used is the **XGB** implementation which supports the GPU acceleration

In [12]:
# load training set
import pandas as pd
import numpy as np

XTrain = pd.read_csv('../../data/staging_data/mispelling_fixed_clean_input_train.csv', sep=',')
YTrain = pd.read_csv('c:/Users/I051796/Projects/CES/data/label.csv', sep=';')

##  Data Preparation

In [13]:
# vectorize the text with TF-IDF transform
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

vectorizer = TfidfVectorizer(strip_accents='ascii')
XTFIDFVectorizedTrain = vectorizer.fit_transform(XTrain['question'])
XTFIDFVectorizedTrain = pd.DataFrame(XTFIDFVectorizedTrain.toarray())
XTFIDFVectorizedTrain.head(2)

,0,1,2,3,4,5,6,7,8,9,...,8062,8063,8064,8065,8066,8067,8068,8069,8070,8071
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# perform a dimensional reduction into 600 dimensional space
# use the same dimensionality than the one provided by fasttext embedding model in the DL context
from sklearn.decomposition import PCA

PCATransform = PCA(n_components=600)
PCAXTrain = pd.DataFrame(PCATransform.fit_transform(XTFIDFVectorizedTrain))

In [19]:
from sklearn.model_selection import train_test_split
# combine input features from above text vectorization with text characteristics features (../../data/staging_data/text_extracted_features.csv)
extractedFeatures = pd.read_csv('../../data/staging_data/text_extracted_features.csv', sep=',')
mergedXTrain = PCAXTrain.join(extractedFeatures, lsuffix='', rsuffix='')

# split into train and test
mergedXTrain, mergedXTest, YTrain, YTest = train_test_split(mergedXTrain, YTrain, test_size=0.15, random_state=42)

mergedXTrain.head(5)

,0,1,2,3,4,5,6,7,8,9,...,quandCount,quoiCount,commentCount,avecCount,etCount,ouCount,interrogativeIndex,interrogativeWordIndex,drugWordIndex,drugInterrogativeDistance
1188,0.143400,-0.123155,-0.047099,-0.025021,-0.036193,-0.031144,0.030765,-0.049536,0.067703,0.059016,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
4252,-0.105870,0.037012,-0.089505,-0.021650,0.211299,0.115834,0.115130,-0.028592,-0.000775,-0.025345,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
447,-0.080297,-0.038696,-0.039266,-0.152591,0.059328,-0.133384,0.115167,-0.044083,0.044766,0.083318,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
2926,0.065797,0.110953,0.150924,-0.027691,0.049147,-0.028346,-0.076531,0.058098,0.105040,0.103262,...,0,0,0,0,0,0,NaN,NaN,0.0,NaN
3733,-0.074550,0.025568,0.039872,0.003111,0.030488,-0.033056,-0.081783,0.068414,-0.017495,0.020567,...,0,0,0,0,0,0,NaN,NaN,3.0,NaN


## Hyper-parameter Search

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# run a grid search with key parameter value candidates
# max_depth
# min_child_weight
# n_estimators
# early stopping
# learning_rate (eta)

grid_parameters = {
    'max_depth':(4,6,8),
    "min_child_weight" :(2, 5, 10),
    "learning_rate" :(0.05, 0.1)
}

gbm = xgb.XGBClassifier(
    objective = "multi:softprob",
    n_estimators=100,
    eval_metric ="mlogloss",
    n_jobs=2,    
    tree_method='gpu_hist',
    n_gpus=1)

gridSearch = GridSearchCV(
    estimator=gbm,
    fit_params = None,
    param_grid = grid_parameters,
    cv=4,
    verbose=1)

gridSearch.fit(mergedXTrain, YTrain.intention)

Fitting 4 folds for each of 18 candidates, totalling 72 fits


d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

## Final Model Fit

In [7]:
gbm.fit(mergedXTrain, YTrain.intention, early_stopping_rounds=30, eval_set=[(mergedXTrain,YTrain.intention)], eval_metric='merror')

[0]	validation_0-merror:0.477063
Will train until validation_0-merror hasn't improved in 30 rounds.
[1]	validation_0-merror:0.435146
[2]	validation_0-merror:0.416386
[3]	validation_0-merror:0.399384
[4]	validation_0-merror:0.385021
[5]	validation_0-merror:0.37227
[6]	validation_0-merror:0.362597
[7]	validation_0-merror:0.352338
[8]	validation_0-merror:0.34413
[9]	validation_0-merror:0.334017
[10]	validation_0-merror:0.324637
[11]	validation_0-merror:0.318042
[12]	validation_0-merror:0.308076
[13]	validation_0-merror:0.301627
[14]	validation_0-merror:0.293126
[15]	validation_0-merror:0.287117
[16]	validation_0-merror:0.283893
[17]	validation_0-merror:0.277884
[18]	validation_0-merror:0.27334
[19]	validation_0-merror:0.26909
[20]	validation_0-merror:0.266159
[21]	validation_0-merror:0.262788
[22]	validation_0-merror:0.257365
[23]	validation_0-merror:0.25502
[24]	validation_0-merror:0.250623
[25]	validation_0-merror:0.244321
[26]	validation_0-merror:0.238898
[27]	validation_0-merror:0.235

[236]	validation_0-merror:0.006888
[237]	validation_0-merror:0.006888
[238]	validation_0-merror:0.006449
[239]	validation_0-merror:0.006302
[240]	validation_0-merror:0.006156
[241]	validation_0-merror:0.006156
[242]	validation_0-merror:0.006156
[243]	validation_0-merror:0.005863
[244]	validation_0-merror:0.005863
[245]	validation_0-merror:0.005716
[246]	validation_0-merror:0.005716
[247]	validation_0-merror:0.005569
[248]	validation_0-merror:0.005569
[249]	validation_0-merror:0.005423
[250]	validation_0-merror:0.005423
[251]	validation_0-merror:0.00513
[252]	validation_0-merror:0.00513
[253]	validation_0-merror:0.00513
[254]	validation_0-merror:0.004983
[255]	validation_0-merror:0.004983
[256]	validation_0-merror:0.004983
[257]	validation_0-merror:0.004983
[258]	validation_0-merror:0.004983
[259]	validation_0-merror:0.004837
[260]	validation_0-merror:0.00469
[261]	validation_0-merror:0.00469
[262]	validation_0-merror:0.004543
[263]	validation_0-merror:0.004543
[264]	validation_0-merror

KeyboardInterrupt: 

In [ ]:
pd.Series(gbm.feature_importances_).describe()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(gbm.feature_importances_)
gbm.feature_importances_

In [ ]:
YPredicted = gbm.predict(mergedXTest)
YTrue = YTest.intention

In [ ]:
%load '../utils/vizu.py'

In [ ]:
%matplotlib inline

import sklearn
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(YTrue, YPredicted)

print(sklearn.metrics.classification_report(YTrue, YPredicted))

plt.figure(figsize=(20,20))

plot_confusion_matrix(cnf_matrix, normalize=False, classes = np.unique(YTrain['intention']))

plt.show()


In [ ]:
Fit on whole training set